## Convolutional Neural Network Trained on Graph Representations of Molecules for Predicting Toxicity

(with stratified k-fold cross validation)

In [1]:
import numpy as np
import pandas
import networkx as nx
from rdkit import Chem
import tensorflow as tf
from tensorflow import keras
from pysmiles import read_smiles
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

2023-01-11 14:36:53.956267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# first obtain a list of smiles:

In [2]:
descriptors = pandas.read_csv("data/rdkit_descriptors.csv") # load in rdkit descriptors from file
desp_array = np.array(descriptors)
x_smiles = list(desp_array[:,1]) # isolate smiles of 554 molecules from the rray of rdkit descriptors
# x_smiles

In [3]:
# then convert them to Graphs - represented as adjacency matrices:

In [4]:
def smiles_to_graph(smiles: str) -> nx.Graph:
    # First, use the RDKit library to parse the SMILES string into a molecular graph
    mol = Chem.MolFromSmiles(smiles)
    
    # Next, convert the RDKit molecular graph into a NetworkX graph
    G = nx.Graph()
    for i in range(mol.GetNumAtoms()):
        # Add the atomic number and explicit valence as attributes to the node
        G.add_node(i, atomic_num=mol.GetAtomWithIdx(i).GetAtomicNum(),
                      explicit_valence=mol.GetAtomWithIdx(i).GetExplicitValence())
        
    for i in range(mol.GetNumBonds()):
        # Add the bond type as an attribute to the edge
        G.add_edge(mol.GetBondWithIdx(i).GetBeginAtomIdx(),
                   mol.GetBondWithIdx(i).GetEndAtomIdx(),
                   bond_type=mol.GetBondWithIdx(i).GetBondType())
    
    return G

In [5]:
x_graphs = []
for smile in x_smiles:
    graph = smiles_to_graph(smile)
    x_graphs.append(graph)
# x_graphs

In [6]:
# obtain the y data - binarized toxicity labels:

In [7]:
original = pandas.read_csv("data/fathead_minnow_dataset.csv")
lc50 = original["LC50_(mg/L)"]

lc50_binary_list = []
for value in lc50:
    if value > 0.5:
        lc50_binary_list.append(0) # not high toxicity
    elif value <= 0.5:
        lc50_binary_list.append(1)

y = np.array(lc50_binary_list)
# y

In [8]:
# need to then process the data as the matrices are all different shapes - pad smaller matrices with zeros:

In [9]:
def preprocess_data(graphs):
    # Convert each NetworkX graph into a NumPy array
    adjacency_matrices = [nx.to_numpy_array(G) for G in graphs]

    # Find the maximum shape of the adjacency matrices
    max_shape = max([matrix.shape for matrix in adjacency_matrices])

    # Pad the smaller matrices with zeros to make them the same shape as the largest matrix
    padded_matrices = [np.pad(matrix, pad_width=[(0, max_shape[0] - matrix.shape[0]), (0, max_shape[1] - matrix.shape[1])], mode='constant', constant_values=0) for matrix in adjacency_matrices]

    # Stack the padded arrays into a single 3D tensor
    adjacency_matrix = np.stack(padded_matrices, axis=0)
    return adjacency_matrix

In [10]:
x_processed = preprocess_data(x_graphs)

In [11]:
# obtain shape of the x data
x_processed.shape

(554, 33, 33)

In [13]:
# build the model

seed = 42 # manually set the seed value as it is needed for the kfold set up

keras.backend.clear_session()
tf.random.set_seed(seed)  

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed) # sets up the k-fold, n_splits = 5 so its 5-fold cross validation
cvscores = [] # to record accuracy scores from each run

X = x_processed # name your x dataset "X"

for train, test in kfold.split((tf.expand_dims(X, axis=-1)), y): # this loop sets a new test/train split for each iteration in the loop
    
    model_cnn = keras.models.Sequential([
        keras.layers.Conv2D(64, 7, activation = 'relu', padding = 'same',
                            input_shape = [33,33,1]),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same'),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same'),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation = 'relu'),
        keras.layers.Dense(64, activation = 'relu'),
        keras.layers.Dense(10, activation = 'softmax')
    ])

    model_cnn.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'nadam',
                  metrics = ['accuracy'])
    
    model_cnn.fit(X[train], y[train], epochs=30, verbose=0)
    
    # these lines here record all the scores
    scores = model_cnn.evaluate(X[test], y[test], verbose=0) 
    print("%s: %.2f%%" % (model_cnn.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

accuracy: 90.99%
accuracy: 92.79%
accuracy: 93.69%
accuracy: 90.09%
accuracy: 92.73%
92.06% (+/- 1.32%)
